In [73]:
import pandas as pd
import numpy as np
import clickhouse_connect
from elasticsearch import Elasticsearch

In [54]:
def get_pinpp_of_defendant(limit=300):
    host = "play.clickhouse.com" 
    port = 443
    user = "explorer" 
    password = ""
    query = f"""
SELECT OriginAirportID FROM ontime
WHERE toDate(FlightDate) >= '2022-03-31'
ORDER BY FlightDate DESC
limit {limit}
    """
    client = clickhouse_connect.get_client(
        host=host,
        port=port,  
        user=user,
        password=password,
        secure=True  
    )
    query = query
    result = client.query(query)
    client.close()
    

    return result.result_rows

result=get_pinpp_of_defendant()

df_def=pd.DataFrame(result, columns=['pinfl'])
df_def['jinoyatchimi']=1

pinfl_list_to_check = list(df_def['pinfl'])

In [55]:
def get_pinpp_of_normal_humans(not_get_pinfl_list, limit=100000):
    host = "play.clickhouse.com" 
    port = 443
    user = "explorer" 
    password = ""
    excluded_pinpps = ", ".join([f"'{pinpp}'" for pinpp in not_get_pinfl_list])
    
    query = f"""
    SELECT OriginAirportID
    FROM ontime
    WHERE OriginAirportID NOT IN ({excluded_pinpps})
    LIMIT {limit}
    """

    client = clickhouse_connect.get_client(
        host=host,
        port=port,
        user=user,
        password=password,
        secure=True 
    )
    
    result = client.query(query)
    client.close() 
    return result.result_rows

result=get_pinpp_of_normal_humans(pinfl_list_to_check, limit=100000)

df_normal=pd.DataFrame(result, columns=['pinfl'])
df_normal['jinoyatchimi']=0

In [56]:
df = pd.concat([df_def, df_normal], ignore_index=True).sample(frac = 1)
pinpp_list_overall = list(df['pinfl'])

In [76]:
import clickhouse_connect

def get_data_by_pinpp(pinpp_list_overall=pinpp_list_overall, chunk_size=100, limit=10000):
    host = "play.clickhouse.com"
    port = 443
    user = "explorer"
    password = ""

    client = clickhouse_connect.get_client(
        host=host,
        port=port,
        user=user,
        password=password,
        secure=True
    )

    chunks = [pinpp_list_overall[i:i + chunk_size] for i in range(0, len(pinpp_list_overall), chunk_size)]
    
    result = []

    for chunk in chunks:
        formatted_pinpp_list = ", ".join([f"'{pinpp}'" for pinpp in chunk])
        query = f"""
        SELECT *
        FROM ontime
        WHERE OriginAirportID IN ({formatted_pinpp_list})
        LIMIT {limit}  -- Natijani limitlash
        """
        partial_result = client.query(query)
        result.extend(partial_result.result_rows)

    client.close()

    return result
